# Tapis-Pylauncher - Simple Example

Carlos del-Castillo-Negrete (cdelcastillo21@gmail.com)

### NOTES

- Note this notebooks is meant to be run with a bash kernel. If one is not installed for your jupyter instance install it using pip. See more instructions at https://pypi.org/project/bash_kernel/.
and 

- This notebook assumes you have setup tapis in your current environment and have access to the pylauncher Tapis application. See the Setup notebook for more info. Make sure to run `tapis auth init --interactive` to referesh sessions in seperate terminal window if the Tapis session gets stale.

In [3]:
# Activate target environment
export env_name=demo
conda activate $env_name

(demo) 

: 1

## Configuring

In [9]:
export example=simple

Let us configure a basic run of the shell app. First lets set some variable to their full paths for convenience.

In [10]:
# Set base directory of where git hub project has been checked out.
export nb_dir=$(dirname pwd)
export base_dir=$(realpath $nb_dir/..)
export app_dir=$base_dir/app
export job_configs=$app_dir/job_configs
export ini=$app_dir/project.ini

echo $base_dir

/home/jupyter/MyData/WebinarDemo/tapis-pylauncher


Lets create a data for sending inputs and downloading data from job runs:

In [11]:
export data_dir=$nb_dir/data
export inputs_dir=$nb_dir/data/inputs
export down_dir=$nb_dir/data/downloads
mkdir -p $data_dir; mkdir -p $inputs_dir; mkdir -p $down_dir

Lets view the inputs that will be passed to our job:

In [12]:
echo "Example $example Input Contents: "; ls -l $job_configs/$example

Example simple Input Contents: 
total 2
-rw-rw-r--+ 1 jupyter G-816877 188 Jan 26 06:04 jobs_list.csv
-rwxrwxr-x+ 1 jupyter G-816877 934 Jan 26 07:20 main.sh
-rw-rw-r--+ 1 jupyter G-816877 342 Jan 26 17:21 simple.json


## Basic Run

### Jobs to be Run

Note how there is a static `jobs_list.csv` file in the simple example

In [7]:
cat $job_configs/$example/jobs_list.csv

2,echo "Pre Process" >> run_1.log ;./main.sh 1 >> run_1.log ;echo "Post Process" >> run_1.log
2,echo "Pre Process" >> run_2.log ;./main.sh 2 >> run_2.log ;echo "Post Process" >> run_2.log


### Submitting Job

To submit job, first zip job directory up and then submit with appopriate json file. 

In [7]:
# Zip job inputs directory directory to send to job
rm -f $inputs_dir/$example.zip
zip -r $inputs_dir/$example.zip $job_configs/$example/*

# Submit job - Note how we explicitly pass location of job json file and ini file as well
tapis jobs submit -F $job_configs/$example/$example.json --ini $ini

(demo) (demo) (demo)   adding: home/jupyter/MyData/WebinarDemo/tapis-pylauncher/app/job_configs/shell_demo/cleanup.sh (deflated 41%)
  adding: home/jupyter/MyData/WebinarDemo/tapis-pylauncher/app/job_configs/shell_demo/generator.py (deflated 56%)
  adding: home/jupyter/MyData/WebinarDemo/tapis-pylauncher/app/job_configs/shell_demo/generator.sh (deflated 41%)
  adding: home/jupyter/MyData/WebinarDemo/tapis-pylauncher/app/job_configs/shell_demo/main.sh (deflated 39%)
  adding: home/jupyter/MyData/WebinarDemo/tapis-pylauncher/app/job_configs/shell_demo/post_process.sh (deflated 39%)
  adding: home/jupyter/MyData/WebinarDemo/tapis-pylauncher/app/job_configs/shell_demo/pre_process.sh (deflated 40%)
  adding: home/jupyter/MyData/WebinarDemo/tapis-pylauncher/app/job_configs/shell_demo/setup.sh (deflated 46%)
  adding: home/jupyter/MyData/WebinarDemo/tapis-pylauncher/app/job_configs/shell_demo/shell_demo.json (deflated 44%)
(demo) total 3
drwxrwxr-x+ 3 jupyter G-816877 4096 Jan 26 07:55 shell_

: 1

### Monitoring Job

Once we've kicked off the job we can use a variety of methods to check in on how its doing using the tapis CLI. 

First we can start by using the show operation to get all the job info regarding the job we just submitted. 

Note how the latest job is grabbed from the tapis job list by grabbing the first column of the first row of the table returned by `tapis jobs list`.

In [8]:
# Show status of job just submitted - should be latest job in job list
export latest_job=$(tapis jobs list | awk 'FNR == 4 {print $2}')
tapis jobs show $latest_job

(demo) (demo) +--------------------+---------------------------------------------------------+
| Field              | Value                                                   |
+--------------------+---------------------------------------------------------+
| accepted           | 2022-01-26T16:06:19.260Z                                |
| appId              | pylauncher-test-0.0.1                                   |
| appUuid            | 6724058001663463916-242ac118-0001-005                   |
| archive            | True                                                    |
| archiveOnAppError  | True                                                    |
| archivePath        | clos21/archive/jobs/job-378526c8-e0c0-43c5-afdd-a483523 |
|                    | 8bfa2-007                                               |
| archiveSystem      | designsafe.storage.default                              |
| blockedCount       | 0                                                       |
| created     

: 1

Note how right after submitting, the job is in the STAGING_JOB status. There are several job statuses that a job can cycle through. After waiting a bit, the job will start running:

In [13]:
tapis jobs show $latest_job

usage: tapis jobs show [-h] [-f {json,shell,table,value,yaml}] [-c COLUMN]
                       [--noindent] [--prefix PREFIX] [--max-width <integer>]
                       [--fit-width] [--print-empty] [-H URL] [-z TOKEN]
                       [--no-verify] [-T]
                       JOB_UUID
tapis jobs show: error: the following arguments are required: JOB_UUID



: 1

In [11]:
tapis job show $latest_job

+--------------------+---------------------------------------------------------+
| Field              | Value                                                   |
+--------------------+---------------------------------------------------------+
| accepted           | 2022-01-26T16:06:19.260Z                                |
| appId              | pylauncher-test-0.0.1                                   |
| appUuid            | 6724058001663463916-242ac118-0001-005                   |
| archive            | True                                                    |
| archiveOnAppError  | True                                                    |
| archivePath        | clos21/archive/jobs/job-378526c8-e0c0-43c5-afdd-a483523 |
|                    | 8bfa2-007                                               |
| archiveSystem      | designsafe.storage.default                              |
| blockedCount       | 0                                                       |
| created            | 2022-

: 1

### Checking Job Outputs

At which point we can check on the jobs outputs 

In [94]:
tapis job outputs list $latest_job

+----------------------------------------------------+----------------+--------+
| name                                               | lastModified   | length |
+----------------------------------------------------+----------------+--------+
| 2                                                  | 37 seconds ago |      0 |
| __pycache__                                        | 37 seconds ago |   4096 |
| cleanup.sh                                         | 35 minutes ago |    554 |
| generator.py                                       | 2 minutes ago  |   1387 |
| generator.sh                                       | 13 minutes ago |    832 |
| hostlist3.py                                       | 44 seconds ago |  13823 |
| jobs_list.json                                     | 37 seconds ago |    760 |
| launch.py                                          | 44 seconds ago |    455 |
| logs                                               | 37 seconds ago |   4096 |
| main.sh                   

: 1

In [52]:
tapis job outputs download --cwd  $latest_job run_1.log

+-------------+-------+
| Field       | Value |
+-------------+-------+
| downloaded  | 0     |
| skipped     | 0     |
| messages    | 1     |
| elapsed_sec | 5     |
+-------------+-------+
(demo) 

: 1

In [51]:
!pwd

pwd
/home/jupyter/MyData/WebinarDemo/tapis-pylauncher/app/job_configs
(demo) 

: 1